In [1]:
import csv
import json
import pandas as pd
import re
import os

#Tách tên .wav + duration từu .csv gốc tạo thành 1 tuple
def extract_file_info(csv_path):
    """
    Input: File .csv có các trường như file,text,duration
    Output: List tuple (file_name, duration)
    """
    file_info_list = []
    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            file_path = row['file']
            duration = float(row['duration'])
            file_name = file_path.split('/')[-1]
            file_info_list.append((file_name, duration))

    return file_info_list

def get_duration(file_name, file_info_list):
    """
    Trả về duration của file_name từ file_info_list.
    
    Parameters:
    file_name (str): Tên file cần tìm (vd: '542.wav').
    file_info_list (list): Danh sách các tuple chứa tên file và duration.
    Returns:
    float: Thời lượng của file nếu tìm thấy, None nếu không tìm thấy.
    """
    for file, duration in file_info_list:
        if file == file_name:
            return duration
    return None

def extract_and_save_csv(input_csv_path, output_csv_path):
    """
    Trích xuất các trường cần thiết từ file CSV ban đầu và lưu vào file CSV mới.

    Parameters:
    input_csv_path (str): Đường dẫn tới file CSV ban đầu.
    output_csv_path (str): Đường dẫn tới file CSV mới sẽ lưu.

    Returns:
    None
    """
    with open(input_csv_path, newline='', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = ['file', 'text', 'duration']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

        writer.writeheader()
        for row in reader:
            name_wav = f"replace/{extract_filename(row['file'])}"
            new_row = {
                'file': name_wav,
                'text': row['text_bartpho'],
                'duration': row['duration']
            }
            writer.writerow(new_row)

def remove_punctuation(text):
    return re.sub(r'[^\w\s]',' ', text)

def clean_text(text):
    """ Chuẩn hóa từ bằng cách chuyển về viết thường và xóa bỏ dấu cách (space) dư thừa

    Args:
        text(str): Câu đầu vào
    Returns:
        (str): Câu sau khi được chuẩn hóa
    """
    text = text.lower()
    text = remove_punctuation(text)
    text = ' '.join(text.split())
    return text 

def extract_filename(file_path):
    """
    Input:
        /mnt/driver/pdnguyen/studen_annoted/data_telesale_extract_10_dir/extra_0/wavs/1620.wav
    Output:
        1620.wav
    """
    return os.path.basename(file_path)

In [3]:
from datasets import load_dataset

cv_17 = load_dataset("mozilla-foundation/common_voice_17_0", "en", split="train", streaming=True)

/home/pdnguyen/.conda/envs/infer_fcm/lib/python3.9/site-packages/datasets/load.py:1478: FutureWarning: The repository for mozilla-foundation/common_voice_17_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_17_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [7]:
print(cv_17)

IterableDataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    n_shards: 28
})


In [16]:
# Lặp qua các phần tử trong cv_17 và lấy trường sentence
with open("sentences.txt", "w", encoding="utf-8") as f:
    for example in cv_17:
        sentence = example['sentence']
        text_clean = clean_text(sentence)
        f.write(text_clean + "\n")

Reading metadata...: 1101170it [00:36, 30026.41it/s]
Exception ignored from cffi callback <function SoundFile._init_virtual_io.<locals>.vio_read at 0x7f82e74a4c10>:
Traceback (most recent call last):
  File "/home/pdnguyen/.conda/envs/infer_fcm/lib/python3.9/site-packages/soundfile.py", line 1244, in vio_read
    try:
KeyboardInterrupt: 
